# Audio Construction

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import pickle
from pydub import AudioSegment
from pydub.playback import play
import moviepy.editor as mp

## Functions

In [2]:
# function to construct audio
def construct_audio(quantized_impacts, instrument_paths, audio_save_path, audio_load_path=None, video_length:float=None):
    
    # quantized impacts and instrument paths should both have length n_limbs
    assert len(quantized_impacts) == len(instrument_paths)
        
    # if audio_load_path provided, construct audio on top of original audio (we don't have this yet)
    if audio_load_path:
        base_audio = AudioSegment.from_wav(audio_load_path)
        
#         # if audio file provided is shorter than video length, then concatenate .wav file
#         frames = base_audio.getnframes()
#         rate = base_audio.getframerate()
#         temp_audio = base_audio
#         while (frames / rate) < video_length:
#             base_audio = base_audio + temp_audio
#             frames = base_audio.getnframes()
        
    # otherwise, construct blank audio file
    elif video_length:
        base_audio = AudioSegment.silent(duration=(1000*video_length))
    else:
        raise Exception('audio_load_path or video_length must be provided')
    
    # quantized_impacts is a list of numpy arrays
    # each array corresponds to a limb and contains timestamps in seconds of the impact points
    # each limb has a corresponding instrument sound in instrument_paths
    # instrument sounds should occur in the audio file at every time stamp in the respective array
    for impacts, instrument_path in zip(quantized_impacts, instrument_paths):
        impact_sound = AudioSegment.from_wav(instrument_path)
        
#         print(impacts)
        print(instrument_path)
        
        # for each time stamp in array, make sound in file & combine sound file with base_audio
        for time in impacts:
            beat = AudioSegment.silent(duration=(1000*time)) + impact_sound
            quite_beat = beat - 9
            base_audio = base_audio.overlay(quite_beat)
    
    final_sound = base_audio
    
    # save audio to audio_save_path
    final_sound.export(audio_save_path, format=audio_save_path[-3:])

In [3]:
# stripping sound from video and re-saving
def strip_audio(video_load_path, audio_save_path, video_save_path=None):
    video = mp.VideoFileClip(video_load_path)
    video.audio.write_audiofile(audio_save_path)
    
    if video_save_path != None:
        video_no_audio = video.without_audio()
        video_no_audio.write_videofile(video_save_path)

In [4]:
# combining video and audio
def combine_audio_video(video_load_path, audio_load_path, combined_save_path, fps=30):
    video = mp.VideoFileClip(video_load_path)
    audio = mp.AudioFileClip(audio_load_path)
    
    video.audio = audio
    video.write_videofile(combined_save_path, fps=fps, audio_codec='aac')

## Strip Audio (run once)

In [7]:
# strip audio for all videos
video_original_dir = os.path.join('video', 'original')
audio_original_dir = os.path.join('audio', 'original')
if not os.path.exists(audio_original_dir):
    os.makedirs(audio_original_dir)

# iterate over all videos
vid_info_df = pd.read_csv(os.path.join('video', 'video_info.csv'), index_col='vid_name')
for vid_name in vid_info_df.index:
    if vid_name != 'group':
        print(50 * '-')
        print(vid_name)
        
        video_original_path = os.path.join(video_original_dir, f'{vid_name}.mov')
        audio_original_path = os.path.join(audio_original_dir, f'{vid_name}.wav')
        
        strip_audio(video_original_path, audio_original_path)

--------------------------------------------------
a1
MoviePy - Writing audio in audio/original/a1.wav


MoviePy - Done.
--------------------------------------------------
a2
MoviePy - Writing audio in audio/original/a2.wav


MoviePy - Done.
--------------------------------------------------
e1
MoviePy - Writing audio in audio/original/e1.wav


MoviePy - Done.
--------------------------------------------------
e2
MoviePy - Writing audio in audio/original/e2.wav


MoviePy - Done.
--------------------------------------------------
e3
MoviePy - Writing audio in audio/original/e3.wav


MoviePy - Done.
--------------------------------------------------
j1
MoviePy - Writing audio in audio/original/j1.wav


MoviePy - Done.
--------------------------------------------------
j2
MoviePy - Writing audio in audio/original/j2.wav


MoviePy - Done.
--------------------------------------------------
u1
MoviePy - Writing audio in audio/original/u1.wav


MoviePy - Done.
--------------------------------------------------
u2
MoviePy - Writing audio in audio/original/u2.wav


MoviePy - Done.


## Construct Audio and Video

In [5]:
# construct audio and video
vid_name = 'j1'

# load quantized impacts
quant_load_dir = os.path.join('data', 'quantized_impacts')
with open(os.path.join(quant_load_dir, f'{vid_name}_quantized_impacts.pkl'), 'rb') as f:
    quantized_impacts = pickle.load(f)

# get instrument paths
instrument_dir = os.path.join('audio', 'instruments')
instrument_names = ['hihat.wav', 'cowbell.wav', 'kick.wav', 'snare.wav']
instrument_paths = [os.path.join(instrument_dir, name) for name in instrument_names]

# get load directory
audio_original_dir = os.path.join('audio', 'original')
audio_original_path = os.path.join(audio_original_dir, f'{vid_name}.wav')

# create save directory
audio_output_dir = os.path.join('audio', 'output')
if not os.path.exists(audio_output_dir):
    os.makedirs(audio_output_dir)
audio_output_path = os.path.join(audio_output_dir, f'{vid_name}.wav')

construct_audio(quantized_impacts, instrument_paths, audio_output_path, audio_load_path=audio_original_path)

# https://github.com/jiaaro/pydub/issues/209
#   getting permission denied error when try to test with play(AudioSegment.from_wav(audio_save_path)); need to look @ changing TEMPDIR
# play(AudioSegment.from_wav(audio_save_path)) """

# combine audio and video
video_original_dir = os.path.join('video', 'original')
video_original_path = os.path.join(video_original_dir, f'{vid_name}.mov')
video_output_dir = os.path.join('video', 'output')
video_output_path = os.path.join(video_output_dir, f'{vid_name}.mp4')
if not os.path.exists(video_output_dir):
    os.makedirs(video_output_dir)

combine_audio_video(video_original_path, audio_output_path, video_output_path)

audio/instruments/hihat.wav
audio/instruments/cowbell.wav
audio/instruments/kick.wav
audio/instruments/snare.wav
Moviepy - Building video video/output/j1.mp4.
MoviePy - Writing audio in j1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video/output/j1.mp4



Moviepy - Done !
Moviepy - video ready video/output/j1.mp4


## Run on All Videos

In [6]:
# run audio and video construction for all videos
vid_info_df = pd.read_csv(os.path.join('video', 'video_info.csv'), index_col='vid_name')

# directories
quant_load_dir = os.path.join('data', 'quantized_impacts')
audio_original_dir = os.path.join('audio', 'original')
audio_output_dir = os.path.join('audio', 'output')
if not os.path.exists(audio_output_dir):
    os.makedirs(audio_output_dir)

video_original_dir = os.path.join('video', 'original')
video_output_dir = os.path.join('video', 'output')
if not os.path.exists(video_output_dir):
    os.makedirs(video_output_dir)

# get instrument paths
instrument_dir = os.path.join('audio', 'instruments')
instrument_names = ['hihat.wav', 'cowbell.wav', 'kick.wav', 'snare.wav']
instrument_paths = [os.path.join(instrument_dir, name) for name in instrument_names]

# iterate over all videos
for vid_name in vid_info_df.index:
    if vid_name != 'group':
        print(50 * '-')
        print(vid_name)
        
        # load quantized impacts
        with open(os.path.join(quant_load_dir, f'{vid_name}_quantized_impacts.pkl'), 'rb') as f:
            quantized_impacts = pickle.load(f)
        
        # construct audio
        audio_original_path = os.path.join(audio_original_dir, f'{vid_name}.wav')
        audio_output_path = os.path.join(audio_output_dir, f'{vid_name}.wav')
        construct_audio(quantized_impacts, instrument_paths, audio_output_path, audio_load_path=audio_original_path)

        # combine audio and video
        video_original_path = os.path.join(video_original_dir, f'{vid_name}.mov')
        video_output_path = os.path.join(video_output_dir, f'{vid_name}.mp4')
        combine_audio_video(video_original_path, audio_output_path, video_output_path)

--------------------------------------------------
a1
audio/instruments/hihat.wav
audio/instruments/cowbell.wav
audio/instruments/kick.wav
audio/instruments/snare.wav
Moviepy - Building video video/output/a1.mp4.
MoviePy - Writing audio in a1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video/output/a1.mp4



Moviepy - Done !
Moviepy - video ready video/output/a1.mp4
--------------------------------------------------
a2
audio/instruments/hihat.wav
audio/instruments/cowbell.wav
audio/instruments/kick.wav
audio/instruments/snare.wav
Moviepy - Building video video/output/a2.mp4.
MoviePy - Writing audio in a2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video/output/a2.mp4



Moviepy - Done !
Moviepy - video ready video/output/a2.mp4
--------------------------------------------------
e1
audio/instruments/hihat.wav
audio/instruments/cowbell.wav
audio/instruments/kick.wav
audio/instruments/snare.wav
Moviepy - Building video video/output/e1.mp4.
MoviePy - Writing audio in e1TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready video/output/j2.mp4
--------------------------------------------------
u1
audio/instruments/hihat.wav
audio/instruments/cowbell.wav
audio/instruments/kick.wav
audio/instruments/snare.wav
Moviepy - Building video video/output/u1.mp4.
MoviePy - Writing audio in u1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video/output/u1.mp4



Moviepy - Done !
Moviepy - video ready video/output/u1.mp4
--------------------------------------------------
u2
audio/instruments/hihat.wav
audio/instruments/cowbell.wav
audio/instruments/kick.wav
audio/instruments/snare.wav
Moviepy - Building video video/output/u2.mp4.
MoviePy - Writing audio in u2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video/output/u2.mp4



Moviepy - Done !
Moviepy - video ready video/output/u2.mp4
